In [1]:
import json
import re
from pathlib import Path
import shutil

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================
# Pasta onde estão os arquivos já limpos pelo script anterior
INPUT_DIR = Path('data/json/cleaned_factors')

# Subdiretórios de destino
DIR_INDETERMINADOS = INPUT_DIR / 'has_undetermined'
DIR_IDENTIFICADOS = INPUT_DIR / 'identified_factors'

def remover_tag_contribuiu(lista_fatores):
    """
    Remove a string "(contribuiu)" e variações de espaçamento dos itens da lista.
    Ex: "Atitude (contribuiu)" -> "atitude"
    """
    lista_limpa = []
    for item in lista_fatores:
        # Regex explicações:
        # \s* -> Zero ou mais espaços antes
        # \(            -> Parêntese abrindo literal
        # \s* -> Espaços opcionais dentro
        # contribuiu    -> A palavra alvo
        # \s* -> Espaços opcionais
        # \)            -> Parêntese fechando literal
        # $             -> Fim da string (garante que é um sufixo)
        novo_item = re.sub(r'\s*\(\s*contribuiu\s*\)$', '', item, flags=re.IGNORECASE)
        lista_limpa.append(novo_item.strip())
    return lista_limpa

def processar_separacao():
    if not INPUT_DIR.exists():
        print(f"Diretório de entrada não encontrado: {INPUT_DIR}")
        return

    # Cria os subdiretórios se não existirem
    DIR_INDETERMINADOS.mkdir(parents=True, exist_ok=True)
    DIR_IDENTIFICADOS.mkdir(parents=True, exist_ok=True)

    arquivos = list(INPUT_DIR.glob("*.json"))
    print(f"Analisando {len(arquivos)} arquivos em '{INPUT_DIR}'...")

    cont_indeterminados = 0
    cont_identificados = 0

    for arquivo in arquivos:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)

            # Verificação de segurança se a chave existe
            if 'conteudo' in dados and 'fatores_contribuintes' in dados['conteudo']:
                fatores = dados['conteudo']['fatores_contribuintes']

                # Verifica se ALGUM fator na lista contém "(indeterminado)"
                # Usamos 'indeterminado)' para pegar casos como "(indeterminado)" ou "(indeterminados)"
                tem_indeterminado = any('indeterminado)' in f.lower() for f in fatores)

                if tem_indeterminado:
                    # CASO 1: Vai para a pasta de indeterminados
                    # Salvamos sem alterar mais nada (preservando o estado para análise)
                    destino = DIR_INDETERMINADOS / arquivo.name
                    with open(destino, 'w', encoding='utf-8') as f_out:
                        json.dump(dados, f_out, ensure_ascii=False, indent=4)
                    cont_indeterminados += 1

                else:
                    # CASO 2: Vai para a pasta de identificados (casos limpos)
                    # AQUI aplicamos a regra de remover "(contribuiu)"
                    fatores_limpos = remover_tag_contribuiu(fatores)

                    # Atualiza o JSON
                    dados['conteudo']['fatores_contribuintes'] = fatores_limpos

                    destino = DIR_IDENTIFICADOS / arquivo.name
                    with open(destino, 'w', encoding='utf-8') as f_out:
                        json.dump(dados, f_out, ensure_ascii=False, indent=4)
                    cont_identificados += 1

            else:
                print(f"Arquivo {arquivo.name} pulado (estrutura inválida).")

        except Exception as e:
            print(f"Erro ao processar {arquivo.name}: {e}")

    print("-" * 40)
    print("Processamento Concluído!")
    print(f"Arquivos com Indeterminados: {cont_indeterminados} -> salvos em {DIR_INDETERMINADOS}")
    print(f"Arquivos Identificados (Limpos): {cont_identificados} -> salvos em {DIR_IDENTIFICADOS}")
    print("-" * 40)
    print("Nota: Os arquivos originais na raiz 'cleaned_factors' foram mantidos.")

if __name__ == "__main__":
    # Pequeno teste de unidade para a regex
    print("--- Teste de Remoção ---")
    teste = ["atitude (contribuiu)", "supervisão (Contribuiu)", "fator normal"]
    print(f"Antes: {teste}")
    print(f"Depois: {remover_tag_contribuiu(teste)}")
    print("-" * 40)

    processar_separacao()

--- Teste de Remoção ---
Antes: ['atitude (contribuiu)', 'supervisão (Contribuiu)', 'fator normal']
Depois: ['atitude', 'supervisão', 'fator normal']
----------------------------------------
Analisando 1410 arquivos em 'data\json\cleaned_factors'...
----------------------------------------
Processamento Concluído!
Arquivos com Indeterminados: 417 -> salvos em data\json\cleaned_factors\has_undetermined
Arquivos Identificados (Limpos): 993 -> salvos em data\json\cleaned_factors\identified_factors
----------------------------------------
Nota: Os arquivos originais na raiz 'cleaned_factors' foram mantidos.
